In [24]:
import nltk #Do tokens
import re
from tabulate import tabulate
import numpy as np
import pandas as pd

In [25]:
sourceCode=open("sourcecode.txt").read()  #Reading Source Code
sourceCodeTokens=nltk.wordpunct_tokenize(sourceCode) 
print(sourceCodeTokens)


['c', '=', "'", 'X', "'", ';', 'switch', '(', 'c', ')', '{', 'case', "'", 'N', "'", ':', 'y', '=', '30', ';', 'break', ';', 'case', "'", 'X', "'", ':', 'y', '=', '60', ';', 'break', ';', 'default', ':', 'y', '=', '0', ';', 'break', ';', '}']


In [26]:
tokens_names=[] #Give each token its type
for token in sourceCodeTokens:
    if re.findall('switch|case|break|deafult',token):
        tokens_names.append('keyword')
    elif re.findall('[0-9]+',token):
        tokens_names.append('digit')
    elif re.findall('=',token):
        tokens_names.append('Operator')
    elif re.findall("^\'[A-Za-z]\'",token):
        tokens_names.append('string')
    elif re.findall("^[A-Za-z][0-9A-Za-z]*",token):
        tokens_names.append('identifier')
    elif  re.findall('\'|(|)|;|{|}|:',token):
        if token=='\'':
            tokens_names.append('single quote')
        elif token=='(':
            tokens_names.append('Left parenthesis')
        elif token==')':
            tokens_names.append('Right parenthesis')
        elif token=='{':
            tokens_names.append('Left brace')
        elif token=='}':
            tokens_names.append('Right brace')
        elif token==';':
            tokens_names.append('Semicolone')
        elif token==':':
            tokens_names.append('Colone')

In [27]:
headers = ['Token','Type']   

TokensTable = zip(sourceCodeTokens,tokens_names)

print(tabulate(TokensTable, headers=headers)) #PRINTING PHASE1 OUTPUT



Token    Type
-------  -----------------
c        identifier
=        Operator
'        single quote
X        identifier
'        single quote
;        Semicolone
switch   keyword
(        Left parenthesis
c        identifier
)        Right parenthesis
{        Left brace
case     keyword
'        single quote
N        identifier
'        single quote
:        Colone
y        identifier
=        Operator
30       digit
;        Semicolone
break    keyword
;        Semicolone
case     keyword
'        single quote
X        identifier
'        single quote
:        Colone
y        identifier
=        Operator
60       digit
;        Semicolone
break    keyword
;        Semicolone
default  identifier
:        Colone
y        identifier
=        Operator
0        digit
;        Semicolone
break    keyword
;        Semicolone
}        Right brace


In [28]:
#SWITCH-CASE CFG
CFG_LHS=['Start','Sdash','Cdash','Ddash','statement','expression','character','digit'] 
CFG_RHS=[['statementSdash'] ,['switch(character){CdashDdash}'] , 
        ['caseexpression:statementbreak;' ,'CdashCdash'],
        ['default:statementbreak;'] ,['character=expression;'],['\'character\'','digit'],['y','X','c','N'],['0','30','60']]

In [29]:
#DEFINIG LISTS WE NEED
stack=[] #daaa lbykon feeh l tokens l shghala aliha 
parserStack=[] #PARSER STACK OF LR BE PRINTED
inputStack=[] #INPUT STACK OF LR TO BE PRINTED
action=[] # ACTION OF LR TO BE PRINTED
tempStack=[] # Has copy of souceCodeTokens list
for i in sourceCodeTokens: #copying tokens
    tempStack.append(i)
tempStack.append("$")

parserStack.append(" ") #FIRST CELL OF THE parser stack

iterations=1 #btkhlene amshi men akher l stack  fe len(stack)-iterations w bfdl tzed lehad ma awsl le awlo
counter=0 # 3dad l tokens lgowa l str
#str,str1,str2 do the same function for diff purpose
str="" 
str1=""
str2=""


for temp in tempStack: #INITIALIZE INPUT OF INPUT STACK
    str1=str1+temp
inputStack.append(str1)
tempStack.pop(0)

for token in sourceCodeTokens:
    str1=""
    stack.append(token)   #SHIFT 

    for temp in tempStack: #UPDATE WHAT INSIDE THE INPUT STACK
        str1=str1+temp
    inputStack.append(str1)
    tempStack.pop(0)

    for temp in stack: #UPDATE WHAT INSIDE THE PRASER STACK
        str2=str2+temp
    parserStack.append(str2)
    str2=''
    action.append(f'SHIFT {token}') #DO SHIFT

    while True:
        flag=False #TRUE IF THERE IS A REDUCE
        str=stack[len(stack)-iterations]+str
        counter+=1 
        for index in range(len(CFG_RHS)): #LOOPING ACROSS THE CFG RHS
            for i in range(len(CFG_RHS[index])): 
                if str==CFG_RHS[index][i]:
                    for i in range(counter):
                        stack.pop()  
                    stack.append(CFG_LHS[index])
                    for temp in stack: #for parser stack after reduce
                        str2=str2+temp
                    parserStack.append(str2)
                    str2=''
                    action.append(f'REDUCE {stack[len(stack)-1]}->{str}') #REDUCE
                    inputStack.append(str1) #NO CHANGE
                    #initializing variables
                    counter=0 
                    str=""
                    iterations=1
                    flag=True
                    break
            if flag==True: #DONOT CONTINUE SEARCHING INSIDE CFG
                break
        if flag== False: # mdkhlsh wala 3mal pop wala append le haga (NO REDUCE)
            if len(stack)-(iterations+1)>=0: # case1: there are still tokens inside the stack
                iterations+=1
            else: # case2: mfish haga tani fel stack a loop 3liha fa harg3 a shift haga
               str=""
               iterations=1
               counter=0
               break


print(stack)




['Start']


In [30]:
if stack[0]=='Start' and inputStack[len(inputStack)-1]=="$":
    parserStack.append(stack[0])
    action.append('ACCEPTED')
    print('ACCEPTED')
else:
    action.append('REJECTED')
    print('REJECTED')

ACCEPTED


In [31]:
headers = ['ParserStack','InputStack','Action']   

table = zip(parserStack,inputStack,action)

print(tabulate(table, headers=headers))


ParserStack                                                           InputStack                                                                  Action
--------------------------------------------------------------------  --------------------------------------------------------------------------  --------------------------------------------
                                                                      c='X';switch(c){case'N':y=30;break;case'X':y=60;break;default:y=0;break;}$  SHIFT c
c                                                                     ='X';switch(c){case'N':y=30;break;case'X':y=60;break;default:y=0;break;}$   REDUCE character->c
character                                                             ='X';switch(c){case'N':y=30;break;case'X':y=60;break;default:y=0;break;}$   SHIFT =
character=                                                            'X';switch(c){case'N':y=30;break;case'X':y=60;break;default:y=0;break;}$    SHIFT '
character='                 

In [32]:
statment=[]
body=[]
switch=[]
cc=[]
case1=[]
case2=[]
default=[]
tata=[]
mylist=[]
mylistup=[]
for x in range(0,len(sourceCodeTokens)):
    if sourceCodeTokens[x]!="switch":
        statment.append(sourceCodeTokens[x]) # ba7ot el code kolo fyh list 3alshan a2asemo w a7oto fyh lists gowa
    else:
        switchindex=x #ba7ot el index beta3 el switch fyh variable 3alshan abda2 meno ba3d keda
        break
# for x in sourceCodeTokens:
#     d=sourceCodeTokens.count("case") # ba3raf 3adad el cases ely 3andy
for c in range(switchindex,len(sourceCodeTokens)):
    body.append(sourceCodeTokens[c]) #bageeb el code kolo men 3and el switch
for c in range(switchindex,switchindex+4):
    switch.append(sourceCodeTokens[c])
for c in range(switchindex-2,len(body)):
    cc.append(body[c]) # bageeb el code kolo men ta7t el switch
for x in statment:
    if x==';': # removing the ;
        statment.remove(x)
for x in cc: 
    if x=='{': # removing {
        cc.remove(x)
for x in cc:
    if x=='}': #removing }
        cc.remove(x)
for x in range(0,len(cc)):
    if cc[x] !="break":
        tata.append(cc[x+1])  #keda e7na hena 5adna awel statment men el cases of hya dynamic
    else:
        t=x
        break
for x in range(t+2,len(cc)):
    if cc[x] !="break":
        tata.append(cc[x+1]) #keda ana gebt el case el 2
    else:
        d=x # ba3raf ana we2ft feeen 3alshan abda2 el mara ely menha 
        break
for x in range(d+2,len(cc)-1):
    tata.append(cc[x]) # gebt a5er case ely hya el default
for x in range(0,len(tata)):
    if tata[x]!='case':
        mylist.append(tata[x])
tata.remove(':')
tata.remove('default')
tata.remove(':')
mylist.remove('default')
for x in range(0,len(mylist)):
    if mylist[x]!=';':
        mylistup.append(mylist[x])
# for x in range(0,len(mylist)):
#     if mylist[x]==':':
#         mylist.remove(mylist[x])
print(statment)
print(switch)
print(mylist)
# all=[case1,case2,default]
# print(all)

['c', '=', "'", 'X', "'"]
['switch', '(', 'c', ')']
["'", 'N', "'", ':', 'y', '=', '30', ';', 'break', "'", 'X', "'", ':', 'y', '=', '60', ';', 'break', ':', 'y', '=', '0', ';', 'break']


In [33]:
z=''
for x in range(0,len(statment)): # ba-search 3ala el equal w ba7otaha fyh el awal 
    if statment[x]=='=':
        z='='
if z=='=':
    statment.remove("=")
    statment.insert(0,'=')
print(z)
print(statment)

=
['=', 'c', "'", 'X', "'"]


In [34]:
statmentstr_organized=''
statmento=[]
# for x in range(0,len(statment)):
#         statment.insert(0,'(')
statment.insert(0,'(')
statment.insert(len(statment),')')# ba7it brackets 3alshan el library betegberny asta3'dem brackets fah bazthaa
for x in range(0,len(statment)):
        if statment[x]!="'":# bensheel el ' 3alshan 3ayzha men 3'erha 3alshan el syntax tree
                statmento.append(statment[x])
for x in statmento:
        statmentstr_organized+=x+' '
print(statmentstr_organized)

( = c X ) 


In [35]:
switch.remove('switch')
switch.remove('(')
switch.remove(')') # basheel el switch w el brackets men el list
print(switch)

['c']


In [36]:
switchstr=' '
for x in switch:
        switchstr+=' '+x # ba7awel el value lyh list 
print(switchstr)

  c


In [37]:
mylist_organized1=[]
organized1=[]
cond1=[]
for x in range(0,len(mylist)):
        if mylist[x+1]!='break':
                if mylist[x]!=';':# 3alshan a5od kol 7aga gowa el case le7ad el break
                        mylist_organized1.append(mylist[x])
        else:
                a1=x
                break
mylist_organized1.remove(':')# bashel el :
for x in range(0,len(mylist_organized1)):
        if mylist_organized1[x]!="'":# bashel el '
                organized1.append(mylist_organized1[x])
switch_value1=organized1[0] # ba5od el value el awal 3alshan dah el value beta3 el case
organized1.remove(switch_value1) # ba3melo remove 3alshan mesh 3ayzo
z=''
for x in range(0,len(organized1)): # ba-search 3ala el equal w ba7otaha fyh el awal 
    if organized1[x]=='=':
        z='='
if z=='=':
    organized1.remove("=")
    organized1.insert(0,'=') # bashoof feen el = w ba3deen a5alyha fel awal 
cond1=[switch_value1]
cond1.insert(0,switchstr)
cond1.insert(0,'=')
print(switch_value1)
print(organized1)

N
['=', 'y', '30']


In [38]:
bracketo=' '
braktito=[]
organized1.insert(0,'(')
organized1.insert(len(organized1),')')# ba7it brackets 3alshan el library betegberny asta3'dem brackets fah bazthaa
for x in range(0,len(organized1)):
        if organized1[x]!="'":# bensheel el ' 3alshan 3ayzha men 3'erha 3alshan el syntax tree
               braktito.append(organized1[x])
for x in braktito:
        bracketo+=' '+x #ba7awelha string
print(bracketo) # el value beta3 el condition ely fyh case1

  ( = y 30 )


In [39]:
# hena ba3mel el el condition beta3 awel case bazawed el 7arf beta3 el switch w el 7arf beta3 el case w el = sign
# w ba3den 7awetaha string 3alshan teb2a 3ala el form
condtio1=' '
condtion1=[]
condtion1=[switch_value1]
condtion1.insert(0,switchstr)
condtion1.insert(0,'=')
condtion1.insert(0,'(')
condtion1.insert(len(condtion1),')')
for x in condtion1:
        condtio1+=' '+x
print(condtio1)

  ( =   c N )


In [40]:
mylist_organized2=[]
organized2=[]
cond2=[]
for x in range(a1+2,len(mylist)):
        if mylist[x+1]!='break':
                if mylist[x]!=';':
                        mylist_organized2.append(mylist[x])
        else:
                a2=x
                break
mylist_organized2.remove(':')
for x in range(0,len(mylist_organized2)):
        if mylist_organized2[x]!="'":
                organized2.append(mylist_organized2[x])
switch_value2=organized2[0]
organized2.remove(switch_value2)
for x in range(0,len(organized2)): # ba-search 3ala el equal w ba7otaha fyh el awal 
    if organized2[x]=='=':
        z='='
if z=='=':
    organized2.remove("=")
    organized2.insert(0,'=') # bashoof feen el = w ba3deen a5alyha fel awal 
print(organized2)
print(switch_value2)

['=', 'y', '60']
X


In [41]:
bracketo1=' '
braktito1=[]
organized2.insert(0,'(')
organized2.insert(len(organized2),')')# ba7it brackets 3alshan el library betegberny asta3'dem brackets fah bazthaa
for x in range(0,len(organized2)):
        if organized1[x]!="'":# bensheel el ' 3alshan 3ayzha men 3'erha 3alshan el syntax tree
               braktito1.append(organized2[x])
for x in braktito1:
        bracketo1+=' '+x #ba7awelha string
print(bracketo1) # el value beta3 el condition ely fyh case1

  ( = y 60 )


In [42]:
# hena ba3mel el el condition beta3 awel case bazawed el 7arf beta3 el switch w el 7arf beta3 el case w el = sign
# w ba3den 7awetaha string 3alshan teb2a 3ala el form
condtio2=' '
condtion2=[]
condtion2=[switch_value2]
condtion2.insert(0,switchstr)
condtion2.insert(0,'=')
condtion2.insert(0,'(')
condtion2.insert(len(condtion2),')')
for x in condtion2:
        condtio2+=' '+x
print(condtio2)

  ( =   c X )


In [43]:
myliststr3=' '
mylist_default=[]
for x in range(a2+2,len(mylist)):
        if mylist[x+1]!='break':# bageeb a5er case ely hya el default
                if mylist[x]!=';':
                        mylist_default.append(mylist[x])
        else:
                break
mylist_default.remove(':')
print(mylist_default)


['y', '=', '0']


In [44]:
myliststr3=' '
for x in range(0,len(mylist_default)): # ba-search 3ala el equal w ba7otaha fyh el awal 
    if mylist_default[x]=='=':
        z='='
if z=='=':
    mylist_default.remove("=")
    mylist_default.insert(0,'=') # bashoof feen el = w ba3deen a5alyha fel awal 
print(mylist_default)
mylist_default.insert(0,'(')
mylist_default.insert(len(mylist_default),')')# ba7it brackets 3alshan el library betegberny asta3'dem brackets fah bazthaa
for x in mylist_default:
        myliststr3+=' '+x
print(myliststr3)

['=', 'y', '0']
  ( = y 0 )


In [45]:
if stack[0]=='Start' and inputStack[len(inputStack)-1]=="$":
    z='(code'+statmentstr_organized+'(switch '+switchstr+' (body ( case '+switch_value1+' '+bracketo+' break ) (case '+switch_value2+' '+bracketo1+' break) (default '+myliststr3+' break))))'
    omar=nltk.Tree.fromstring(z)
    omar.draw()
else:
    print('REJECTED')